In [1]:
import pandas as pd

In [2]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

(CVXPY) Mar 19 02:24:28 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Mar 19 02:24:28 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


In [3]:
file = r'crypto_adj_close.csv'
data = pd.read_csv(file, index_col="Date", parse_dates=True)

In [4]:
# drop the USDS33039-USD and SUI20947-USD

data = data.drop(columns=['USDS33039-USD', 'SUI20947-USD'])

# drop the NaN values
data = data.dropna()

In [5]:
data

,BTC-USD,ETH-USD,USDT-USD,BNB-USD,USDC-USD,XRP-USD,ADA-USD,SOL-USD,DOGE-USD,TRX-USD,...,DAI-USD,WTRX-USD,STETH-USD,LINK-USD,LEO-USD,XLM-USD,WSTETH-USD,HBAR-USD,SHIB-USD,BCH-USD
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-05 00:00:00+00:00,39400.585938,2664.831055,1.000275,384.873352,0.999371,0.754900,0.865672,89.647842,0.124996,0.060428,...,1.000046,0.060396,2637.910889,14.022286,5.343326,0.180100,2827.397705,0.210644,0.000025,293.979858
2022-03-06 00:00:00+00:00,38419.984375,2555.037354,1.000244,375.011200,0.999851,0.725994,0.823752,84.542183,0.120766,0.058920,...,0.999439,0.058955,2549.643555,13.338135,5.372933,0.173367,2719.219971,0.202762,0.000024,283.082794
2022-03-07 00:00:00+00:00,38062.039062,2497.771240,1.000123,381.344635,0.999134,0.723287,0.797379,82.130066,0.117105,0.058680,...,0.999825,0.058566,2491.440674,12.712721,5.381678,0.170453,2653.854248,0.206794,0.000023,274.935089
2022-03-08 00:00:00+00:00,38737.269531,2576.747559,1.000282,381.958557,1.000260,0.720977,0.801250,82.254761,0.117029,0.060544,...,1.000307,0.060757,2572.363281,13.077511,5.642929,0.172986,2735.103760,0.204938,0.000023,284.369629
2022-03-09 00:00:00+00:00,41982.925781,2729.783447,1.000294,393.555756,1.000450,0.767236,0.849293,88.202065,0.121588,0.060841,...,1.000205,0.060668,2725.346436,14.015894,5.805223,0.189024,2894.686523,0.212987,0.000024,308.348907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-10 00:00:00+00:00,78532.000000,1861.151367,0.999680,532.121887,1.000043,2.018294,0.672597,118.291748,0.154060,0.229486,...,1.000006,0.229769,1870.114502,12.693139,9.747614,0.242302,2247.683350,0.189435,0.000012,331.645325
2025-03-11 00:00:00+00:00,82862.210938,1919.844971,0.999776,551.474548,0.999932,2.166916,0.722587,125.263947,0.164166,0.224426,...,0.999964,0.224822,1916.963379,13.099883,9.882753,0.255025,2307.500244,0.195807,0.000012,339.491882
2025-03-12 00:00:00+00:00,83722.359375,1908.982666,0.999687,569.894836,0.999767,2.240755,0.735444,126.583244,0.172381,0.222995,...,0.999969,0.222525,1905.024658,13.482376,9.709141,0.259589,2278.803955,0.200960,0.000012,353.958008


In [37]:
mu = expected_returns.mean_historical_return(data)
s = risk_models.sample_cov(data)

In [38]:
ef = EfficientFrontier(mu, s)

In [44]:
import pandas as pd
import matplotlib.pyplot as plt
from pypfopt import plotting


fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True, show_fig=False)

# Customize the plot
ax.set_title('Efficient Frontier')
ax.set_xlabel('Volatility (Standard Deviation)')
ax.set_ylabel('Expected Return')

plt.show()

OSError: 'seaborn-deep' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

In [39]:
max_sharpe = ef.max_sharpe()

In [ ]:
cleaned_weights = ef.clean_weights()

OrderedDict([('BTC-USD', 0.0),
             ('ETH-USD', 0.0),
             ('USDT-USD', 0.0),
             ('BNB-USD', 0.0),
             ('USDC-USD', 0.0),
             ('XRP-USD', 0.11191),
             ('ADA-USD', 0.0),
             ('SOL-USD', 0.0),
             ('DOGE-USD', 0.0),
             ('TRX-USD', 0.01658),
             ('TON-USD', 0.0),
             ('MATIC-USD', 0.0),
             ('DOT-USD', 0.0),
             ('LTC-USD', 0.0),
             ('AVAX-USD', 0.0),
             ('WBTC-USD', 0.1228),
             ('DAI-USD', 0.0),
             ('WTRX-USD', 0.34284),
             ('STETH-USD', 0.0),
             ('LINK-USD', 0.0),
             ('LEO-USD', 0.40587),
             ('XLM-USD', 0.0),
             ('WSTETH-USD', 0.0),
             ('HBAR-USD', 0.0),
             ('SHIB-USD', 0.0),
             ('BCH-USD', 0.0)])

In [32]:
for i in cleaned_weights:
    if cleaned_weights[i] > 0:
        print(i, cleaned_weights[i])

XRP-USD 0.11191
TRX-USD 0.01658
WBTC-USD 0.1228
WTRX-USD 0.34284
LEO-USD 0.40587


In [35]:
ef.portfolio_performance(verbose=True)

Expected annual return: 24.2%
Annual volatility: 34.7%
Sharpe Ratio: 0.64


(0.24165496391900657, 0.34653183671782173, 0.6396380950691655)